### First Regression (Turnover intention ~ unfair treatment x neg. reciprocity)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import math

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

### Read in SOEP Data:
- vp : 2005 data : main variables of interest: questions on negative reciprocity
- wp: 2006 data : main variables of interest: question on perceived recognition for work
- xp: 2007 data : main variables of interest: turnover intentions, controls

In [2]:
# define path: insert the path where the SOEP data is stored on your computer here
from pathlib import Path
data_folder = Path(f"C:/Users/max-admin/Desktop/Masterstudium/WiSe_22_23/Research_Module/SOEP-Data/Stata/raw")
# define relevant subsets of SOEP-data
file_names = ['vp', 'wp', 'xp']

file_paths = [data_folder / f"{file_name}.dta" for file_name in file_names]
# some controls are in gen data
file_paths_2 = [data_folder / f"{file_name}gen.dta" for file_name in file_names]

In [3]:
# read in 2005 data for the reciprocity measures
data05 = pd.read_stata(file_paths[0], columns=["pid","hid", "syear","vp12602", "vp12603", "vp12605"]).set_index(['pid', 'hid'])
df_05 = data05.rename(columns={ 'vp12602': 'take_revenge', 'vp12603': 'similar_problems', 'vp12605': 'insult_back'})
df_05.head(3)

,,syear,take_revenge,similar_problems,insult_back
pid,hid,,,,
201,27,2005,[1] Trifft ueberhaupt nicht zu,[5] Skala 1-7,[1] Trifft ueberhaupt nicht zu
203,60313,2005,[2] Skala 1-7,[3] Skala 1-7,[2] Skala 1-7
602,60,2005,[5] Skala 1-7,[4] Skala 1-7,[3] Skala 1-7


In [11]:
# read in 2006 data
# personal = personal advancement
# still includes all unfair treat
data06 = pd.read_stata(file_paths[1], columns=["pid", "hid", "syear", 'wp43b01', 'wp43b02', 'wp43b03', 'wp43b04', 'wp43b05', 'wp43b06', 'wp43b07','wp43b08' ]).set_index(['pid', 'hid'])
df_06 = data06.rename(columns={ 'wp43b01': 'recog_sup', 'wp43b02': 'felt_recog_sup',"wp43b03": "recog_effort",  'wp43b04': 'felt_recog_effort', "wp43b05": "recog_personal", "wp43b06" :"felt_recog_personal" ,"wp43b07": "recog_pay",'wp43b08': 'felt_recog_pay',})
df_06.head(3)

,,syear,recog_sup,felt_recog_sup,recog_effort,felt_recog_effort,recog_personal,felt_recog_personal,recog_pay,felt_recog_pay
pid,hid,,,,,,,,,
201,27,2006,[-2] trifft nicht zu,[-2] trifft nicht zu,[-2] trifft nicht zu,[-2] trifft nicht zu,[-2] trifft nicht zu,[-2] trifft nicht zu,[-2] trifft nicht zu,[-2] trifft nicht zu
203,60313,2006,[1] Ja,[-2] trifft nicht zu,[1] Ja,[-2] trifft nicht zu,[2] Nein,[2] Maessig,[2] Nein,[2] Maessig
602,60,2006,[2] Nein,[2] Maessig,[1] Ja,[-2] trifft nicht zu,[1] Ja,[-2] trifft nicht zu,[1] Ja,[-2] trifft nicht zu


In [12]:
#read in 2007 data
# here left out 'xp8601' for school degree since we have it in another module also 'xp0102' : 'work_satisfaction' for the beginning
#for outcome and all controls
data3= pd.read_stata(file_paths[2], columns=["pid", "hid", "syear", 'xp13101' , 'xp13102', 'xp2701', 'xp7302','xp7202','xp28', 'xp3001']).set_index(['pid', 'hid'])
df_07 = data3.rename(columns= {'xp13101':'gender','xp13102': 'year_birth' ,'xp2701': 'turnover_intention' , 'xp7302': 'wage_lastmonth','xp7202': 'overtime','xp28': 'new_job', 'xp3001': 'reason_new_job'})
df_07.head(3)

,,syear,gender,year_birth,turnover_intention,wage_lastmonth,overtime,new_job,reason_new_job
pid,hid,,,,,,,,
201,27,2007,[2] Weiblich,1926,[-2] trifft nicht zu,-2,-2,[-2] trifft nicht zu,[-2] trifft nicht zu
203,60313,2007,[1] Maennlich,1960,[0] 0% wahrscheinlich,1200,-2,[2] Nein,[-2] trifft nicht zu
602,60,2007,[2] Weiblich,1958,[90] 90% wahrscheinlich,200,-2,[2] Nein,[-2] trifft nicht zu


In [13]:
# read in 2007 data from work module

# adapt path and merge
hours07 = pd.read_stata(file_paths_2[2], columns=["pid","hid", "syear", "xvebzeit", "xpsbil", "nace07", "betr07", "xerwzeit", "xbilzeit"]).set_index(['pid', 'hid'])
work07 = hours07.rename(columns={'xvebzeit': 'working_hours', "xpsbil": "school_degree", "nace07": "sector", "betr07": "firmsize", "xerwzeit": "tenure" , "xbilzeit" : "years_educ"})
work07.head(5)

,,syear,working_hours,school_degree,sector,firmsize,tenure,years_educ
pid,hid,,,,,,,
201,27,2007,-2.0,[1] Hauptschulabschluss,[-2] trifft nicht zu,[-2] trifft nicht zu,-2.000,10.5
203,60313,2007,-2.0,[4] Abitur,[72] Datenverarbeitung und Datenbanken,[11] Selbstaendig-ohne Mitarb.,2.333,18.0
602,60,2007,-2.0,[4] Abitur,[80] Erziehung und Unterricht,[3] 11 bis unter 20,2.500,18.0
604,60,2007,-2.0,[7] Noch kein Abschluss,[-2] trifft nicht zu,[-2] trifft nicht zu,-2.000,-2.0
901,94,2007,36.5,[1] Hauptschulabschluss,[52] Einzelhandel (ohne Handel mit Kraftfahrze...,[6] 20 bis unter 100,6.083,10.5


##### Define Mappings for categoricals

In [8]:
# mapping for reciprocity questions: same scale for all
reciprocity_questions_mapping = {
    '[1] Trifft ueberhaupt nicht zu': 1,
    '[2] Skala 1-7': 2,
    '[3] Skala 1-7': 3,
    '[4] Skala 1-7': 4,
    '[5] Skala 1-7': 5,
    '[6] Skala 1-7': 6,
    '[7] Trifft voll zu': 7,
    '[-1] keine Angabe': -1,
}
## mapping for recognition questions: binary
recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Ja': 1,
    '[2] Nein': 2,
}
# felt recog mapping
felt_recog_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Gar nicht': 1,
    '[2] Maessig': 2,
    '[3] Stark': 3,
    '[4] Sehr stark': 4,
}
# mapping for firmsize -> we need to recode this in a sensible way: jumps are the same: first change: selbstständig to 0
firmsize_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1,
    '[1] Unter  5': 1,
    '[2] 5 bis 10': 2,
    '[3] 11 bis unter 20': 3,
    '[4] bis 90: unter 20': 4,
    '[5] 91-04: 5 bis unter 20': 5,
    '[6] 20 bis unter 100': 6,
    '[7] 100 bis unter 200': 7,
    '[8] bis 98: 20 bis unter 200': 8,
    '[9] 200 bis unter 2000': 9,
    '[10] 2000 und mehr': 10,
    '[11] Selbstaendig-ohne Mitarb.': 0,
}
# mapping new job into binary variable
new_job_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe': -1, 
    '[1] Ja': 1, 
    '[2] Nein': 2,
    '[3] Ja, nach Datenpruefung': 1,
}
# mapping for turnover intention: split up into binary with roughly equal value counts for simplicity: might change that later to categories
turnover_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[0] 0% wahrscheinlich': 2,
    '[10] 10% wahrscheinlich': 1,
    '[20] 20% wahrscheinlich': 1,
    '[30] 30% wahrscheinlich': 1,
    '[40] 40% wahrscheinlich': 1, 
    '[50] 50% wahrscheinlich': 1,
    '[60] 60% wahrscheinlich': 1,
    '[70] 70% wahrscheinlich': 1,
    '[80] 80% wahrscheinlich': 1,
    '[90] 90% wahrscheinlich': 1,
    '[100] 100% wahrscheinlich': 1,
}
# mapping for new job to easier remove negatives
reason_new_job_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1, 
    '[1] Erstmals erwerbstaetig': 1,
    '[2] Wieder erwerbstaetig': 2,
    '[3] Stelle bei neuen Arbeitgeber': 3,  
    '[4] Uerbnommen von Betrieb': 4,
    '[5] Stellenwechsel im Betrieb': 5, 
    '[6] Selbstaendig geworden': 6,
}
# mapping for sectors: only to easier remove negatives
sector_map = {
    "[1] Landwirtschaft und  Jagd": 1,
    "[2] Forstwirtschaft": 2,
    "[5] Fischerei und Fischzucht": 5,
    "[10] Kohlenbergbau, Torfgewinnung": 10,
    "[11] Gewinnung von Erdöl und Erdgas, Erbringung damit verbundener Dienstleistungen": 11,
    "[12] Bergbau auf Uran- und Thoriumerze": 12,
    "[13] Erzbergbau": 13,
    "[14] Gewinnung von Steinen und Erden, sonstiger Bergbau": 14,
    "[15] Herstellung von Nahrungs- und Futtermitteln sowie Getränken": 15,
    "[16] Tabakverarbeitung": 16,
    "[17] Herstellung von Textilien": 17,
    "[18] Herstellung von Bekleidung": 18,
    "[19] Herstellung von Leder und Lederwaren": 19,
    "[20] Herstellung von Holz sowie Holz-, Kork- und Flechtwaren (ohne Herstellung von Möbeln)": 20,
    "[21] Herstellung von Papier, Pappe und Waren daraus": 21,
    '[22] Herstellung von Verlags- und Druckerzeugnissen,  Vervielfältigung von bespielten Ton-, Bild- und Datenträgern': 22,
    "[23] Kokerei, Mineralölverarbeitung, Herstellung und Verarbeitung von Spalt- und Brutstoffen": 23,
    "[24] Herstellung von chemischen Erzeugnissen": 24,
    "[25] Herstellung von Gummi- und Kunststoffwaren": 25,
    "[26] Herstellung von Glas und Glaswaren, Keramik, Verarbeitung von Steinen und Erden": 26,
    "[27] Metallerzeugung und -bearbeitung": 27,
    "[28] Herstellung von Metallerzeugnissen": 28,
    "[29] Maschinenbau": 29,
    "[31] Herstellung von Geräten der Elektrizitätserzeugung, -verteilung u. Ä.": 31,
    "[30] Herstellung von Büromaschinen, Datenverarbeitungsgeräten und -einrichtungen": 30,
    "[32] Rundfunk- und Nachrichtentechnik": 32,
    "[33] Medizin-, Mess-, Steuer- und Regelungstechnik, Optik, Herstellung von Uhren": 33,
    "[34] Herstellung von Kraftwagen und Kraftwagenteilen": 34,
    "[35] Sonstiger Fahrzeugbau": 35,
    "[36] Herstellung von Möbeln, Schmuck, Musikinstrumenten, Sportgeräten, Spielwaren und sonstigen Erzeugnissen": 36,
    "[37] Rückgewinnung": 37,
    "[40] Energieversorgung": 40,
    "[41] Wasserversorgung": 41,
    "[45] Bau": 45,
    "[50] Kraftfahrzeughandel; Instandhaltung und Reparatur von Kraftfahrzeugen; Tankstellen": 50,
    "[51] Handelsvermittlung und Großhandel (ohne Handel mit Kraftfahrzeugen)": 51,
    "[52] Einzelhandel (ohne Handel mit Kraftfahrzeugen und ohne Tankstellen); Reparatur von Gebrauchsgütern": 52,
    "[55] Beherbergungs- und Gaststätten": 55,
    "[60] Landverkehr; Transport in Rohrfernleitungen": 60,
    "[61] Schifffahrt": 61,
    "[62] Luftfahrt": 62,
    "[63] Hilfs- und Nebentätigkeiten für den Verkehr; Verkehrsvermittlung": 63,
    "[64] Nachrichtenübermittlung": 64,
    "[65] Kreditinstitute": 65,
    "[66] Versicherungen (ohne Sozialversicherung)": 66,
    "[67] Mit den Kreditinstituten und Versicherungen verbundene Tätigkeiten": 67,
    "[70] Grundstücks- und Wohnungswesen": 70,
    "[71] Vermietung beweglicher Sachen ohne Bedienungspersonal": 71,
    "[72] Datenverarbeitung und Datenbanken": 72,
    "[73] Forschung und Entwicklung": 73,
    "[74] Erbringung von unternehmensbezogenen Dienstleistungen": 74,
    "[75] Öffentliche Verwaltung, Verteidigung, Sozialversicherung": 75,
    "[80] Erziehung und Unterricht": 80,
    "[85] Gesundheits-, Veterinär- und Sozialwesen": 85,
    "[90] Abwasser- und Abfallbeseitigung und sonstige Entsorgung": 90,
    "[91] Interessenvertretungen sowie kirchliche und sonstige Vereinigungen (ohne Sozialwesen, Kultur und Sport)": 91,
    "[92] Kultur, Sport und Unterhaltung": 92,
    "[93] Erbringung von sonstigen Dienstleistungen": 93,
    "[95] Private Haushalte mit Hauspersonal": 95,					
    "[96] Industrie - ohne weitere Zuordnung": 96,					
    "[97] Handwerk - ohne weitere Zuordnung": 97,					
    "[98] Dienstleistungen ohne weitere Zuordnung": 98,					
    "[99] Exterritoriale Organisationen und Körperschaften": 99,				
    "[100] Produzierendes Gewerbe ohne w.Zuordnung": 100,
    "[-1] keine Angabe": -1,
    '[-2] trifft nicht zu': -2, 
    "[-3] unplausibler Wert": -3,
    "[-4] unzulaessige Mehrfachantwort": -4, 
    "[-5] in Fragebogenversion nicht enthalten": -5,
    "[-6] Fragebogenversion mit geaenderter Filterfuehrung": -6, 
    "[-7] nur in weniger eingeschraenkter Edition verfuegbar": -7,
    "[-8] Frage in diesem Jahr nicht Teil des Frageprogramms": -8,
}
# mapping for school degree: to easier remove negatives
school_degree_mapping = {
    '[-2] trifft nicht zu': -2,
    '[-1] keine Angabe':-1,
    '[1] Hauptschulabschluss': 1,
    '[2] Realschulabschluss': 2,
    '[3] Fachhochschulreife': 3,
    '[4] Abitur': 4,
    '[5] Anderer Abschluss': 5,
    '[6] Ohne Abschluss verlassen': 6,
    '[7] Noch kein Abschluss': 7,
    '[8] Keine Schule besucht': 8,
}
# reversed mapping to redo changes
reversed_mapping_reason = {v: k for k, v in reason_new_job_mapping.items()}
reversed_mapping_sector = {v: k for k, v in sector_map.items()}
reversed_mapping_schoold = {v: k for k, v in school_degree_mapping.items()}

#### Convert categoricals to numerical to drop them easier

In [7]:
## function for recoding values and dropping missing

def recode_categoricals(inputdf):

    merged = inputdf  
    
    # recode Gender variable
    merged['gender'].replace('[2] Weiblich', 2,inplace=True)
    merged['gender'].replace('[1] Maennlich', 1,inplace=True)
    # recode reciprocity variables
    merged[["similar_problems","take_revenge","insult_back"]] = merged[["similar_problems","take_revenge","insult_back"]].apply(lambda x: x.map(reciprocity_questions_mapping))
    # recode recognition variables
    merged[["recog_sup","recog_effort","recog_personal","recog_pay"]] = merged[["recog_sup","recog_effort","recog_personal","recog_pay"]].apply(lambda x: x.map(recog_mapping))
    # recode felt recognition variables
    merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]] = merged[["felt_recog_sup","felt_recog_effort","felt_recog_personal","felt_recog_pay"]].apply(lambda x: x.map(felt_recog_mapping))
    # recode firm size
    merged['firmsize'] = merged['firmsize'].map(firmsize_mapping)
    # recode new job reason variable
    merged['reason_new_job'] = merged['reason_new_job'].map(reason_new_job_mapping)
    # recode job change variable
    merged['new_job']= merged['new_job'].map(new_job_mapping)
    # recode turnover intention variable
    merged['turnover_intention'] = merged['turnover_intention'].map(turnover_mapping)
    # recode sector
    merged['sector'] = merged['sector'].map(sector_map)
    # recode school degree
    merged['school_degree'] = merged['school_degree'].map(school_degree_mapping)
    
    output = merged
    return output

#### Merge Dataframes and apply recoding

In [29]:
# Merge dataframes: a bit tough to read as its nested, merges 4 dataframes: 2005,2006,2007 and 2007gen
allmerged_df = pd.merge(pd.merge(pd.merge(df_05,df_06,on=["pid", "hid"]),work07,on=["pid","hid"]),df_07,on=["pid", "hid"])
recoded = recode_categoricals(allmerged_df).astype('int')

# construct indsutry-relative wage
sector_wage_averages = recoded.groupby('sector')['wage_lastmonth'].mean()
recoded["sector_avg_wage"] = recoded["sector"].map(sector_wage_averages)
recoded["relative_wage"] = recoded["wage_lastmonth"] / recoded["sector_avg_wage"]
# replaces negative values with n.a.n 
recoded = recoded.mask(recoded < 0, np.nan) 
# construct avg reciprocity measure
recoded['avg_rec'] = recoded[['take_revenge', 'similar_problems', 'insult_back']].mean(axis=1)
# construct age, potential experience and age^2
recoded['age'] = 2007 - recoded['year_birth']
recoded["potential_experience"] = pow((recoded["age"] - 18), 2)
recoded["age_squared"] = (recoded["age"] ** 2) / 100
# recode categoricals back to make it better readable
recoded["reason_new_job"] = recoded["reason_new_job"].map(reversed_mapping_reason)
recoded["sector"]=recoded["sector"].map(reversed_mapping_sector)
recoded["school_degree"] = recoded["school_degree"].map(reversed_mapping_schoold)

# transform binary variables with 1 and 2 into 1 and 0
columns_to_transform = ["recog_sup","recog_effort", "recog_pay", "recog_personal" ,"gender", "turnover_intention", "new_job"]

# Iterate over the columns and replace the values 2 with 0
for col in columns_to_transform:
    recoded[col] = recoded[col].replace({2: 0})

# save df somewhere so its not muted when repeatedly executing this cell: Can later transform that into functions
dfnan = recoded
dfnan.head(3)

c:\Anaconda\anaconda3\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,,syear_x,take_revenge,similar_problems,insult_back,syear_y,recog_sup,felt_recog_sup,recog_effort,felt_recog_effort,recog_personal,...,wage_lastmonth,overtime,new_job,reason_new_job,sector_avg_wage,relative_wage,avg_rec,age,potential_experience,age_squared
pid,hid,,,,,,,,,,,,,,,,,,,,,
201,27,2005,1.0,5.0,1.0,2006,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10.782550,NaN,2.333333,81,3969,65.61
203,60313,2005,2.0,3.0,2.0,2006,1.0,NaN,1.0,NaN,0.0,...,1200.0,NaN,0.0,NaN,2078.595376,0.577313,2.333333,47,841,22.09
602,60,2005,5.0,4.0,3.0,2006,0.0,2.0,1.0,NaN,1.0,...,200.0,NaN,0.0,NaN,1746.445431,0.114518,4.000000,49,961,24.01


#### Drop negative values for important columns:
- convert negatives to zeros and N.a.N
- create industry-relative wage and age

#### Drop all respondents who changed their job in the last year
    - Option 1: drop all which changed jobs
    - Option 2: change turnover intention of those who changed jobs and have a new employer to 1.
        - new job + new employer -> turnover intention to 1
        - new job + other reason and initally no turnover intention -> stays at 0

In [32]:
# Option1

dfnan.drop(index=dfnan[dfnan["new_job"] == 1].index , inplace=True)

# Option2

""" def calculate_combined(row):
    if (row['new_job'] == 1 and row['reason_new_job'] == '[3] Stelle bei neuen Arbeitgeber') or row['turnover_intention'] == 1:
        return 1
    elif (row['new_job'] == 1 or row['reason_new_job'] != '[3] Stelle bei neuen Arbeitgeber') and row['turnover_intention'] == 2:
        return 2
    else:
        return 'invalid'

dfnan = dfnan.assign(all_turnover_intentions=df1.apply(calculate_combined, axis=1))
"""

dfnan.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15443 entries, (201, 27) to (8261002, 826103)
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   syear_x               15443 non-null  int32  
 1   take_revenge          15341 non-null  float64
 2   similar_problems      15325 non-null  float64
 3   insult_back           15342 non-null  float64
 4   syear_y               15443 non-null  int32  
 5   recog_sup             8055 non-null   float64
 6   felt_recog_sup        2763 non-null   float64
 7   recog_effort          8128 non-null   float64
 8   felt_recog_effort     2723 non-null   float64
 9   recog_personal        8058 non-null   float64
 10  felt_recog_personal   2985 non-null   float64
 11  recog_pay             8249 non-null   float64
 12  felt_recog_pay        4201 non-null   float64
 13  syear_x               15443 non-null  int32  
 14  working_hours         6340 non-null   float64
 15 

## Regression

- Probit Estimation: TurnoverIntention_{2005} = Constant + Neg-Rec + Unfair + Rec X Unfair + Controls + Error

- As the measure for unfair treatment in the first regression i first used recog_effort -> decide for one later
    - "When I consider all my accomplishments and efforts, the recognition of I've received seems about right to me"